# Create Animation of mri image evolving over menstrual cycle

## Note that this notebook will only work on the Frank server, as i was unable to transfer the original data onto parka (permission denied)

In [1]:
import os
import subprocess

gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
print("Working directory: ", os.getcwd())

import warnings

warnings.filterwarnings("ignore")

import sys

sys_dir = os.path.dirname(os.getcwd())
sys.path.append(sys_dir)
print("Directory added to path: ", sys_dir)

Working directory:  /home/adele/code/my28brains
Directory added to path:  /home/adele/code


In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import nibabel
import numpy as np
from IPython.display import HTML
from nilearn import plotting

import my28brains.my28brains.viz as viz

In [3]:
def show_slices(slices, cmap="gray"):
    """Function to display row of image slices"""
    fig, axes = plt.subplots(1, len(slices))
    for i_slice, one_slice in enumerate(slices):
        im = axes[i_slice].imshow(one_slice.T, cmap=cmap, origin="lower")
    return fig, axes, im

# Make video of Laura's brain for presentation

In [4]:
# creating paths to brain data, one path for each day, contained in nii_paths

nii_paths = []
for day in range(2,31):
    nii_paths.append(f"/home/data/28andme/sub-01/ses-{day:02}/anat/sub-01_ses-{day:02}_T1w.nii.gz")
print(nii_paths)

['/home/data/28andme/sub-01/ses-02/anat/sub-01_ses-02_T1w.nii.gz', '/home/data/28andme/sub-01/ses-03/anat/sub-01_ses-03_T1w.nii.gz', '/home/data/28andme/sub-01/ses-04/anat/sub-01_ses-04_T1w.nii.gz', '/home/data/28andme/sub-01/ses-05/anat/sub-01_ses-05_T1w.nii.gz', '/home/data/28andme/sub-01/ses-06/anat/sub-01_ses-06_T1w.nii.gz', '/home/data/28andme/sub-01/ses-07/anat/sub-01_ses-07_T1w.nii.gz', '/home/data/28andme/sub-01/ses-08/anat/sub-01_ses-08_T1w.nii.gz', '/home/data/28andme/sub-01/ses-09/anat/sub-01_ses-09_T1w.nii.gz', '/home/data/28andme/sub-01/ses-10/anat/sub-01_ses-10_T1w.nii.gz', '/home/data/28andme/sub-01/ses-11/anat/sub-01_ses-11_T1w.nii.gz', '/home/data/28andme/sub-01/ses-12/anat/sub-01_ses-12_T1w.nii.gz', '/home/data/28andme/sub-01/ses-13/anat/sub-01_ses-13_T1w.nii.gz', '/home/data/28andme/sub-01/ses-14/anat/sub-01_ses-14_T1w.nii.gz', '/home/data/28andme/sub-01/ses-15/anat/sub-01_ses-15_T1w.nii.gz', '/home/data/28andme/sub-01/ses-16/anat/sub-01_ses-16_T1w.nii.gz', '/home/da

In [5]:
import os
import nibabel as nib
from PIL import Image

# Set the directory to save the converted images
image_directory = "/home/adele/code/my28brains/results/anims/mri_anim/"

for i_path, path in enumerate(nii_paths):
    img = nib.load(path).get_fdata()
    img = (img / img.max() * 255).astype("uint8")  # Normalize pixel values
    image_path = os.path.join(image_directory, f"mri_day_{i_path:03}.png")
    #brain shifts, need to take different slices for later days.
    if i_path <7 and i_path !=5:
        Image.fromarray(img[:, 50, :]).save(image_path)
    else: 
        Image.fromarray(img[:, 50, :]).save(image_path)
    


In [6]:
# Set the output file name and path
output_file = os.path.join(image_directory, "movie.mp4")

# Set the frame rate of the output movie (frames per second)
frame_rate = 3

# Use ffmpeg to create the movie from the images
os.system(f"ffmpeg -framerate {frame_rate} -pattern_type glob -i '{image_directory}/*.png' -c:v libx264 -pix_fmt yuv420p {output_file}")

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

256

In [7]:
from PIL import Image, ImageDraw, ImageFont



# Set the font size
font_size = 50

# Set the font style
font = ImageFont.truetype("/home/adele/code/my28brains/results/anims/mri_anim/days_gif/DejaVuSerif.ttf", font_size)



for day in range(2,31):
    # Set the text to display
    text = f"Day {day:02}"

    # Get the width and height of the text
    text_width, text_height = font.getsize(text)

    # Create a new image with a white background
    image = Image.new("RGB", (text_width, text_height), color=(255, 255, 255))

    # Add the text to the image
    draw = ImageDraw.Draw(image)
    draw.text((0, 0), text, fill=(0, 0, 0), font=font)

    # Save the image as a .png file
    directory = "/home/adele/code/my28brains/results/anims/mri_anim/days_gif/"
    filename = f"Day_{day:02}.png"
    fullpath = directory + filename
    image.save(fullpath, "PNG")


In [8]:
day_directory = "/home/adele/code/my28brains/results/anims/mri_anim/days_gif/"

# Set the output file name and path
output_file = os.path.join(day_directory, "day_movie.mp4")

# Set the frame rate of the output movie (frames per second)
frame_rate = 3

# Use ffmpeg to create the movie from the images
os.system(f"ffmpeg -framerate {frame_rate} -pattern_type glob -i '{day_directory}/*.png' -c:v libx264 -pix_fmt yuv420p {output_file}")

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

256